### Basic Setting

#### Packages

In [2]:
import pandas as pd 
import csv
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

'en_US.UTF8'

#### Paths

In [37]:
ada_lead_path = 'data/raw/ada_lead_data.csv'
qq_export_path = 'data/raw/QQ_1028_MGA.csv'
ws_path = 'data/raw/Waterstone_Book.csv'
ag_path = 'data/raw/A_Gange_Book.csv'

### Ada Data Processing

There is a dataset from Adagateaway, we have to extract all the customer's information and make sure the final accounts to be transfered is not in Adagateaway.

Ada_df_header_names: 22 cols
['user_id','state','lead_id','name','email','phone','policy_number','premium','source','sold_date','carrier','co_broke','raw_data','lead_timestamp','product_type','Referral New Business','re','company','options','submitter','sub','locker']

#### Reading and Preprocessing

##### Adagateway

In [13]:
# read data
df_ada = pd.read_csv(ada_lead_path, index_col=0)
# drop records without email
df_ada = df_ada.dropna(subset=['email'])
customers_ada_set = set()
for index, row in df_ada.iterrows():
    full_name = row['name'] if isinstance(row['name'], str) else '' 
    email = row['email'].lower() if isinstance(row['email'], str) else ''
    policy_num = row['policy_number'].upper() if isinstance(row['policy_number'], str) else ''
    first_name, last_name = full_name.split(' ')[0].capitalize(), full_name.split(' ')[-1].capitalize()
    customers_ada_set.add((first_name, last_name, email, policy_num))

In [16]:
print('There are {} pieces of customer data from ada'.format(len(customers_ada_set)))

There are 1158 pieces of customer data from ada


##### WaterStone and A Gange

In [17]:
df_ag = pd.read_csv(ag_path)#.dropna(subset=['Policy Number'])
df_ws = pd.read_csv(ws_path)#.dropna(subset=['Policy Number'])

In [18]:
print('There are {} pieces of agenda data'.format(len(df_ag)))
print('There are {} pieces of waterstone data'.format(len(df_ws)))

There are 6345 pieces of agenda data
There are 29387 pieces of waterstone data


#### Data Filtering
Filter the data by features to ensure that all customers are not in the ada data.

In [19]:
overlap_set = set()
df_overlap = pd.DataFrame(columns= df_ag.columns)
df_null = pd.DataFrame(columns= df_ag.columns)

In [20]:
def extract_cust(df, overlap, ada_set):
    #df = df.dropna(subset=['Customer Email'])
    df_null = pd.DataFrame(columns= df.columns)
    df_overlap = pd.DataFrame(columns= df.columns)
    df_valid = pd.DataFrame(columns= df.columns)
    cust = set()
    for index, row in df.iterrows():
        first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
        last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
        email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
        policy_num = row['Policy Number'].upper() if isinstance(row['Policy Number'], str) else ''
        
        if not policy_num or not first_name or not last_name:
            df_null = df_null.append(df.loc[index],  ignore_index = True)
            continue
        if (first_name, last_name, email, policy_num) in ada_set:
            overlap.add((first_name, last_name, email, policy_num))
            df_overlap = df_overlap.append(df.loc[index],  ignore_index = True)
        else:
            customer_info = (first_name, last_name, email, policy_num)
            if customer_info in cust:
                continue
            cust.add(customer_info)
            df_valid = df_valid.append(df.loc[index],  ignore_index = True)
    return cust, df_null, df_overlap, df_valid

def zip_fomatter(zipcode):
    if not zipcode:
        return ''
    main_part = zipcode.split('-')[0]
    if len(main_part) == 4:
        return '0'+zipcode
    return zipcode

In [21]:
customers_ws_set, df_ws_null, df_ws_overlap, df_ws_valid = extract_cust(df_ws, overlap_set, customers_ada_set)
customers_ag_set, df_ag_null, df_ag_overlap, df_ag_valid = extract_cust(df_ag, overlap_set, customers_ada_set)

### QQ Data Processing

#### Reading and Preprocessing

In [38]:
df_qq = pd.read_csv(qq_export_path)
print('There are {} pieces of data from QQ'.format(len(df_qq)))

There are 36939 pieces of data from QQ


#### Data Filtering

In [39]:
customers_qq_set, df_qq_null, df_qq_overlap, df_qq_valid = extract_cust(df_qq, overlap_set, customers_ada_set)

In [40]:
df_qq_valid.head()

,Customer First Name,Customer Last Name,Customer Street Address,Customer State/Province,Customer Zip Code,Customer Email,Customer Phone Number,Customer Number,Policy Number,Policy Premium,Policy Status,CSR On Customer,Agent On Policy,CSR On Policy,Producer,Effective Date,MGA/Broker
0,Tony,Gross,P.O. Box 187,NJ,07849,rmgross@verizon.net,(973) 584-6311,WSDC-03825,KKO0000008839000,"$1,399.00",Cancelled,Linda Sabino,Linda Sabino,Linda Sabino,Linda Sabino,6/26/2022,"K&K Insurance Group, Inc."
1,Tony,Gross,P.O. Box 187,NJ,07849,tonygross1932@gmail.com,(973) 584-6311,WSDC-03825,KKO0000008839000,"$1,399.00",Cancelled,Linda Sabino,Linda Sabino,Linda Sabino,Linda Sabino,6/26/2022,"K&K Insurance Group, Inc."
2,MICHAEL,METZERMACHER,29 BOSWELL AVE,CT,06360-7901,NaN,(860) 912-4182,NaN,K3788820,"$1,064.20",Active,Aaron Culp,Aaron Culp,Aaron Culp,NaN,12/30/2021,NaN
3,MARGO,GRISELL,54 HOP BROOK RD,CT,06804,NaN,(203) 885-2228,WSDC-04433,6081778336331,"$1,340.00",Active,Richard McLaughlin,Franc Kalanderi,Franc Kalanderi,NaN,12/30/2021,NaN
4,Ashley,Zapotosky,122 Maple Ave.,NJ,07057,ashzap12@gmail.com,(201) 446-1709,WSDC-03663,6BRPG0000007214500,$0.00,Pending,NaN,Linda Sabino,Linda Sabino,Linda Sabino,12/27/2021,"K&K Insurance Group, Inc."


This part is to filter out the qq records with agent or CSR or producer who is in the not_wts_agents（given by Xavier）

In [46]:
not_wts_agents = {"Albert Meng","Aaron Culp","Michael Giambrone",\
                  "Michael Coughlin","Richard McLaughlin","Tony Lin",\
                  "Xavier Ke","Franc Kalanderi","Gaurav Garg",\
                  "Hunter Brown","Joshua Grant","Karin D'Eramo"}

In [47]:
df_after_filtered = df_qq_valid[(~df_qq_valid['CSR On Customer'].isin(not_wts_agents))\
                                &(~df_qq_valid['CSR On Policy'].isin(not_wts_agents))\
                               &(~df_qq_valid['Agent On Policy'].isin(not_wts_agents))\
                               &(~df_qq_valid['Producer'].isin(not_wts_agents))]

In [48]:
df_after_filtered_left = df_qq_valid[(df_qq_valid['CSR On Customer'].isin(not_wts_agents))\
                                |(df_qq_valid['CSR On Policy'].isin(not_wts_agents))\
                               |(df_qq_valid['Agent On Policy'].isin(not_wts_agents))\
                               |(df_qq_valid['Producer'].isin(not_wts_agents))]

In [49]:
df_after_filtered.to_csv('data/Output/qq_valid_filtered_0208.csv')

In [50]:
df_after_filtered_left.to_csv('data/Output/qq_not_waterstone_0208.csv')

### Further Processing

#### Customer Information Extraction

In [51]:
'''
To get the customers'info from the dataframe processed above and filter out the records in the adagetaway based on custermer .
Customer Info -> (First Name, Last Name, Customer Number) 
                    eg.('Jennifer', 'Wu', 'wsdc-03761')
'''
def get_customer_info_from_df(df, ada_set, not_waterstone_set = set()):
    df_result = pd.DataFrame(columns= df.columns)
    customers = set()
    overlap_set = set()
    
    for index, row in df.iterrows():
        first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
        last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
        email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
        cust_num =  row['Customer Number'].lower() if isinstance(row['Customer Number'], str) else ''
        if (first_name, last_name, email) not in ada_set:
            customer_info = (first_name, last_name, cust_num)
            if customer_info in customers:
                continue
            if customer_info in not_waterstone_set and customer_info not in overlap_set:
                print(customer_info)
                overlap_set.add(customer_info)
                continue
            customers.add(customer_info)
            df_result = df_result.append(df.loc[index],  ignore_index = True)
            
    return customers, df_result, overlap_set

'''
To get the policy's info from the dataframe and do the filtering.
'''
def get_policy_based_info_from_df(df, ada_set, not_waterstone_set = set()):
    df_result = pd.DataFrame(columns= df.columns)
    customers = set()
    overlap_set = set()
    
    for index, row in df.iterrows():
        first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
        last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
        email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
        policy_num =  row['Policy Number'].lower() if isinstance(row['Policy Number'], str) else ''
        
        if (first_name, last_name, email) not in ada_set:
            customer_info = (first_name, last_name, cust_num)
            if customer_info in not_waterstone_set and customer_info not in overlap_set:
                print(customer_info)
                overlap_set.add(customer_info)
                continue
            customers.add(customer_info)
            df_result = df_result.append(df.loc[index],  ignore_index = True)
            
    return customers, df_result, overlap_set

Get the customer info not in waterstone first.

In [52]:
cust_set_not_in_waterstone, _, _ = get_customer_info_from_df(df_after_filtered_left, customers_ada_set)

Then, get the customers in the waterstone

In [53]:
cust_set_from_waterstone, df_result, overlap_set = get_customer_info_from_df(df_after_filtered, customers_ada_set, cust_set_not_in_waterstone)

('Matthew', 'Dvorozniak', 'wsdc-00239')
('Tara', 'Cummings', 'wsdc-03863')
('Michael', 'Zangoglia', 'wsdc-03364')


##### Compare to the overlap set

In [54]:
df_overlap = pd.DataFrame(columns= df_after_filtered.columns)
df_final_result = pd.DataFrame(columns= df_after_filtered.columns)
for index, row in df_after_filtered.iterrows():
    first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
    last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
    email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
    cust_num =  row['Customer Number'].lower() if isinstance(row['Customer Number'], str) else ''
    
    customer_info = (first_name, last_name, cust_num)
    if customer_info in overlap_set:
        df_overlap = df_overlap.append(df_after_filtered.loc[index],  ignore_index = True)
    else:
        df_final_result = df_final_result.append(df_after_filtered.loc[index],  ignore_index = True)

##### Filter out by State

In [59]:
df_final_result = df_final_result[(df_final_result['Customer State/Province'] == 'NJ') | (df_final_result['Customer State/Province'] == 'NY')]

In [60]:
_, final_result, _ = get_customer_info_from_df(df_final_result, customers_ada_set)

In [61]:
final_result.to_csv('data/final_result.csv')

### Premium Calculation

This part is to calculate the premium for Xavier to estimate.

In [68]:
df_policies_of_waterstone.head()

,Customer First Name,Customer Last Name,Customer Street Address,Customer State/Province,Customer Zip Code,Customer Email,Customer Phone Number,Customer Number,Policy Number,Policy Premium,Policy Status,CSR On Customer,Agent On Policy,CSR On Policy,Producer,Effective Date,MGA/Broker
0,Tony,Gross,P.O. Box 187,NJ,07849,rmgross@verizon.net,(973) 584-6311,WSDC-03825,KKO0000008839000,1399.0,Cancelled,Linda Sabino,Linda Sabino,Linda Sabino,Linda Sabino,2022-06-26 00:00:00,"K&K Insurance Group, Inc."
1,Tony,Gross,P.O. Box 187,NJ,07849,tonygross1932@gmail.com,(973) 584-6311,WSDC-03825,KKO0000008839000,1399.0,Cancelled,Linda Sabino,Linda Sabino,Linda Sabino,Linda Sabino,2022-06-26 00:00:00,"K&K Insurance Group, Inc."
2,Ashley,Zapotosky,122 Maple Ave.,NJ,07057,ashzap12@gmail.com,(201) 446-1709,WSDC-03663,6BRPG0000007214500,0.0,Pending,NaN,Linda Sabino,Linda Sabino,Linda Sabino,2021-12-27 00:00:00,"K&K Insurance Group, Inc."
3,Ashley,Zapotosky,122 Maple Ave.,NJ,07057,info@fitnessinsurance-kk.com,(201) 446-1709,WSDC-03663,6BRPG0000007214500,0.0,Pending,NaN,Linda Sabino,Linda Sabino,Linda Sabino,2021-12-27 00:00:00,"K&K Insurance Group, Inc."
4,Ashley,Zapotosky,122 Maple Ave.,NJ,07057,tiffany.a.haugh@kandkinsurance.com,(201) 446-1709,WSDC-03663,6BRPG0000007214500,0.0,Pending,NaN,Linda Sabino,Linda Sabino,Linda Sabino,2021-12-27 00:00:00,"K&K Insurance Group, Inc."


In [62]:
_, df_policies_of_waterstone, _ = get_policy_based_info_from_df(df_after_filtered, customers_ada_set)

In [63]:
df_policies_of_waterstone.to_csv('data/waterstone_policies_0208.csv')

In [64]:
# change Effective Date from string Type to datetime Type
df_policies_of_waterstone['Effective Date'] = df_policies_of_waterstone['Effective Date'].apply(lambda x: datetime. strptime(x, '%m/%d/%Y'))


In [65]:
# change Premium from string Type to Float Type 
df_policies_of_waterstone['Policy Premium'] = df_policies_of_waterstone['Policy Premium'].apply(lambda x: locale.atof(x.strip("$")))

In [73]:
# filter by date in 2021
waterstone_policies_2021 = df_policies_of_waterstone.loc[(df_policies_of_waterstone['Effective Date'] >= datetime. strptime('12/31/2020', '%m/%d/%Y'))
                     & (df_policies_of_waterstone['Effective Date'] < datetime. strptime('01/01/2022', '%m/%d/%Y'))]

# filter by date in 2020
waterstone_policies_2020 = df_policies_of_waterstone.loc[(df_policies_of_waterstone['Effective Date'] >= datetime. strptime('12/31/2019', '%m/%d/%Y'))
                     & (df_policies_of_waterstone['Effective Date'] < datetime. strptime('01/01/2021', '%m/%d/%Y'))]



In [74]:
df_policies_of_waterstone = df_policies_of_waterstone[(df_policies_of_waterstone['CSR On Customer'] == 'Linda Sabino')\
                                |(df_policies_of_waterstone['CSR On Policy'] == 'Linda Sabino')\
                               |(df_policies_of_waterstone['Agent On Policy'] == 'Linda Sabino')\
                               |(df_policies_of_waterstone['Producer'] == 'Linda Sabino')]

In [75]:
waterstone_policies_2021.groupby(['Policy Status'])['Policy Premium'].sum()

Policy Status
Active       3936905.99
Cancelled     107872.30
Expired         3810.00
Pending            0.00
Voided             0.00
Name: Policy Premium, dtype: float64

In [76]:
waterstone_policies_2020.groupby(['Policy Status'])['Policy Premium'].sum()

Policy Status
Active       2877732.06
Cancelled      10558.52
Expired      3332026.27
Pending            0.00
Voided             0.00
Name: Policy Premium, dtype: float64

In [70]:
waterstone_policies_2021.to_csv('waterstone_policies_2021.csv')

### Policy Based Marking

#### Build policy dataframe

In [25]:
waterstone_processed_policy_records = pd.DataFrame(columns= ['Customer First Name', 'Customer Last Name', 'Customer Number', 'Policy Num', 'Policy Premium', 'Policy Status', 'Agent On Policy'])

In [33]:
waterstone_processed_policy_records.to_csv('1115_2021/processed_policy_records_11152021.csv')

In [36]:
processed_policy_df = pd.read_csv('1115_2021/processed_policy_records.csv')

In [43]:
processed_policy_df = processed_policy_df.dropna(subset=['Customer First Name', 'Customer Last Name']).reset_index(drop = True)

In [44]:
processed_policy_df.to_csv('1115_2021/processed_policy_records.csv')